In [1]:
import ray
import modules.network_extractor as net_extractor
from osmnx import settings
from modules.indices import graph_based as g_index
from osmnx import simplification
#from modules import generalize as generalize

In [2]:
# Change for your own data base path
data_base_path = "/home/geolab/Desktop/Research/data"

# The extractor instance
extractor = net_extractor.NetworkExtractor()
extractor.DATA_BASE_PATH = data_base_path
 
# Custom OSMnx settings
settings.default_crs = "epsg:4326"

# Generalizer instance
#generalizer = generalize.Generalize()

In [12]:
# the city name in lowercase and slug_case where the graphs are located
city_name = "sydney"

In [13]:
# Load graphs if they are already 
g_walk = extractor.load_graph(f'{city_name}/graph/walk_{city_name}')
g_bike = extractor.load_graph(f'{city_name}/graph/bike_{city_name}')
g_drive = extractor.load_graph(f'{city_name}/graph/drive_{city_name}')
g_public = extractor.load_graph(f'{city_name}/graph/public_{city_name}')

In [14]:
print(f"Pedestrian graph for {city_name}")
print(g_walk.number_of_edges())
print(g_walk.number_of_nodes())

Pedestrian graph for sydney
374896
298766


In [15]:
import copy
import networkx as nx
from shapely import ops, LineString
from itertools import chain
import shapely
import osmnx
import networkx as nx
from shapely.geometry import MultiLineString
from shapely.ops import linemerge

ANGLE_THRESHOLD = 10

def grid_candidate(node_id, g: nx.MultiDiGraph | nx.MultiGraph):

    preselect = False
    if g.is_directed():
        incoming_degree = g.in_degree(node_id)
        outgoing_degree = g.out_degree(node_id)
        if incoming_degree + outgoing_degree == 8: # 4 incoming and 4 outgoing 
            preselect = True
    else:
        incoming_degree = g.degree(node_id)
        outgoing_degree = 0
        if incoming_degree + outgoing_degree == 4: # 4 edges
            preselect = True
            
    if preselect:
        incident_edges = list(g[node_id].keys())

        for neighboor_id in incident_edges:
            edge_data = g.get_edge_data(node_id, neighboor_id)
            road_type = "" if "highway" not in edge_data else edge_data["highway"]
            try:
                length = None if "length" not in edge_data else float(edge_data["length"])
            except Exception as ex: length = None

            if road_type is None or length is None: 
                return False
            
            if road_type != "residential" or length >= 300:
                return False

def is_insterstitial(node_id, g: nx.MultiDiGraph | nx.MultiGraph):
    # for directed, an interstitial node has 1 incomming and 1 outgoing edges
    if g.is_directed():
        if g.out_degree(node_id) == 1 and g.in_degree(node_id) == 1:
            return True
        
    # for undirected, an interstitial node exactly 2 edges
    else:
        if g.degree(node_id) == 2:
            return True
    
    # in any othe cases, return false
    return False

def topology_preservation_generalization(input_graph: nx.MultiGraph | nx.MultiDiGraph, max_iterations=10):
    """
    Based on the generalization methodology propsed in DOI: 10.1007/s41109-022-00521-8 .

    It is an iterative algorithm that generalizes a traffic network by eliminating parallel edges,
    self-loops, dead-ends, low-level gridirons, and interstitial nodes (to be implemented).

    Works both for directed and undirected graphs.
    """

    input_g = copy.deepcopy(input_graph)

    is_modified = True
    iterations = 0

    # add aggr_node_number property to all nodes
    for node_id in input_g.nodes():
        input_g._node[node_id]["aggr_node_number"] = 1

    print(f"Initial: {input_g}")
    # Iterate when the graph has been modified (it has not yet converged) or the max iterations has been reached.
    while is_modified and iterations <= max_iterations:
        is_modified = False
        iterations += 1
        print(f"iteration {iterations}")
        print(f"Starting Nodes: {input_g.number_of_nodes()}")
        print(f"Starting Edges: {input_g.number_of_edges()}")
        # 1) Parallel edges
        delete_edges = []
        for node in input_g.nodes():
            for node_to in input_g[node]:

                # select the shortest parallel edge
                if len(list(input_g[node][node_to].keys())) > 1:
                    min_parallel = 99999999
                    min_parallel_key = -1
                    for parallel_edge_key in input_g[node][node_to].keys():
                        data = input_g.get_edge_data(node, node_to, parallel_edge_key, default=None)
                        if data["length"] > min_parallel:
                            min_parallel = data["length"]
                            min_parallel_key = parallel_edge_key
                        
                    for parallel_edge_key in input_g[node][node_to].keys():
                        if parallel_edge_key != min_parallel_key:
                            delete_edges.append([node, node_to, parallel_edge_key])

        if len(delete_edges) > 0:
            is_modified = True
            print("removing parallel edges")
            input_g.remove_edges_from(delete_edges)


        # 2) Self loops
        delete_edges = []
        for u,v,key in input_g.edges(keys=True):
            if u == v:
                # self loop identified
                delete_edges.append([u,v,key])
                
        if len(delete_edges) > 0:
            is_modified = True
            print("removing self loops")
            input_g.remove_edges_from(delete_edges)
        
        # 3) Simplify dead ends
        delete_candidates: dict[any,set] = {}
        # first pass to select candidates
        for node_id in input_g.nodes():
            incident_edges = input_g[node_id]
            if len(incident_edges) <= 1:
                    delete_candidates[node_id] = set()
        
        # second pass to count edges arriving to the candidates
        for node_id in input_g.nodes():
            # Exiting from nodes
            incident_edges = input_g[node_id]
            for v_edge in incident_edges:
                if v_edge in delete_candidates:
                    delete_candidates[v_edge].add(node_id)
                if node_id in delete_candidates:
                    delete_candidates[node_id].add(v_edge)
        
        # If the node is only accessed once, remove it
        delete_nodes = []
        for node_id, related_nodes in delete_candidates.items():
            if len(related_nodes) <= 1:
                delete_nodes.append(node_id)

        if len(delete_nodes) > 0:
            is_modified = True
            print("removing dead ends")
            for delete_node in delete_nodes:
                incident_edges = input_g[delete_node]
                for v_edge in incident_edges:
                    input_g._node[v_edge]["aggr_node_number"] += input_g._node[delete_node]["aggr_node_number"]
                input_g.remove_node(delete_node)

        # 4) Simplify gridiron structures
        # candidates = []
        # candidates = set()
        # for node_id in input_g.nodes():
        #     if grid_candidate(node): candidates.add(node_id)

        # print(f"Candidates: {len(candidates)}")
        # print("finished processing initial candidates")
        # # add the I cases to the candidates
        # # for u,v in input_g.edges():
        # #     print(u,v)
        # #     incident_edges_u = list(input_g[u])
        # #     incident_edges_v = list(input_g[v])
        # #     # if all incident nodes of the edge are in the gridiron, add it as well
        # #     if incident_edges_u in candidates and incident_edges_v in candidates:
        # #         candidates.add(u)
        # #         candidates.add(v)
        
        # # print("Finished 'I' special cases")
        # print("removing grid")
        # # Aggregate clusters of grids individually to propagate and distribute nodes
        # while len(candidates) > 0:
        #     candidate = candidates.pop()

        #     remove_candidates = [candidate]
        #     gridiron_entrances = []
        #     aggregated = 0
        #     visit = [candidate]
        #     visited = set()
        #     # recursively visit the individual grid to extract the aggregated value
        #     while True:
        #         if len(visit) == 0: break # end condition
        #         current_node = visit.pop()
        #         if current_node in visited: continue # already visited node
        #         visited.add(current_node)
        #         remove_candidates.append(current_node)
        #         aggregated += input_g._node[current_node]["aggr_node_number"]
        #         current_edges = list(input_g[current_node])
        #         for edg in current_edges:
        #             if edg in candidates and edg not in visited:
        #                 visit.append(edg)
        #             elif edg not in candidates and edg not in visited:
        #                 visited.add(edg)
        #                 gridiron_entrances.append(edg)
            
        #     # obtained the data from a single grid element recursively.
        #     if len(gridiron_entrances) > 0:
        #         distributed_node_agg = aggregated / len(gridiron_entrances)

        #     # distribute node aggregate value to gridiron entrances
        #     for entrance in gridiron_entrances:
        #         input_g._node[entrance]["aggr_node_number"] += distributed_node_agg

        #     # remove the nodes from candidates and the graph
        #     for to_remove in remove_candidates:
        #         if to_remove in candidates:
        #             candidates.remove(to_remove)
        #         if input_g.has_node(to_remove):
        #             input_g.remove_node(to_remove)
        #             is_modified = True

        # 5) Remove interstitial nodes  
        #input_copy = input_g.copy()
        nodes_to_check = list(input_g.nodes())

        # for each endpoint node, look at each of its successor nodes
        for node_id in nodes_to_check:
            if not is_insterstitial(node_id, input_g):
                continue

            neighbors = list(input_g.neighbors(node_id))

            if len(neighbors) != 2:
                continue  # Just a sanity check

            #print(neighbors, node_id)
            n1, n2 = neighbors

            if input_g.has_edge(node_id, n1):
                u = n1
                v = n2
            else: 
                u = n2
                v = n1


            # Get edge geometries
            keys = list(input_g.get_edge_data(node_id, u).keys())
            if len(keys) > 1:
                continue
            
            edge_1 = list(input_g.get_edge_data(node_id, u).values())[0]
            edge_2 = list(input_g.get_edge_data(v, node_id).values())[0]
            
            # Combine geometries
            merged_geom = linemerge(MultiLineString([edge_1['geometry'], edge_2['geometry']]))
            combined_length = edge_1["length"] + edge_2["length"]

            # Remove node and its edges
            removed_node_count = input_g._node[node_id]["aggr_node_number"]
            input_g.remove_node(node_id)

            # update neighbours node counts - sum 0.5 nodes to each node
            input_g._node[u]["aggr_node_number"] += (removed_node_count / 2)
            input_g._node[v]["aggr_node_number"] += (removed_node_count / 2)

            # Add new edge if not already present
            if input_g.has_edge(u, v):
                continue
            
            new_edge_data = {}
            for attribute in edge_1.keys():
                if attribute == "geometry":
                    new_edge_data["geometry"] = merged_geom
                elif attribute == "length":
                    new_edge_data["length"] = combined_length
                else:
                    if attribute not in edge_1:
                        edge_1[attribute] = None
                    if attribute not in edge_2:
                        edge_2[attribute] = None

                    if edge_1[attribute] == edge_2[attribute]:
                        new_edge_data[attribute] = edge_1[attribute]
                    elif type(edge_1[attribute]) == list:
                        new_edge_data[attribute] = edge_1[attribute].append(edge_2[attribute])
                    elif type(edge_2[attribute]) == list:
                        new_edge_data[attribute] = edge_2[attribute].append(edge_1[attribute])
                    else:
                        new_edge_data[attribute] = edge_1[attribute]
                        
                #TODO manage other attributes
                
            input_g.add_edge(u, v, **new_edge_data)

        print(f"Ending Nodes: {input_g.number_of_nodes()}")
        print(f"Ending Edges: {input_g.number_of_edges()}")

    # Return the simplified graph
    print(f"Final: {input_g}")
    return input_g

In [130]:
#simple = named_streets_generalization(g_walk, is_directed=False)
#extractor.save_as_shp(simple, f'{city_name}/shp/simplified_named_{city_name}', line_graph=True)

In [ ]:
from modules import utils
tolerance = 10 / utils.DEG_CONVERT
merged_intersections = osmnx.simplification.consolidate_intersections(g_drive, tolerance=tolerance, rebuild_graph=True)


/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/osmnx/simplification.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = gdf_nodes.buffer(tolerance).union_all()
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/osmnx/simplification.py:640: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = node_clusters.centroid


In [16]:
general = topology_preservation_generalization(g_walk, max_iterations=20)

Initial: MultiGraph with 298766 nodes and 374896 edges
iteration 1
Starting Nodes: 298766
Starting Edges: 374896
removing parallel edges
removing dead ends
Ending Nodes: 139424
Ending Edges: 203453
iteration 2
Starting Nodes: 139424
Starting Edges: 203453
removing dead ends
Ending Nodes: 117507
Ending Edges: 181065
iteration 3
Starting Nodes: 117507
Starting Edges: 181065
removing dead ends
Ending Nodes: 114219
Ending Edges: 177535
iteration 4
Starting Nodes: 114219
Starting Edges: 177535
removing dead ends
Ending Nodes: 113536
Ending Edges: 176741
iteration 5
Starting Nodes: 113536
Starting Edges: 176741
removing dead ends
Ending Nodes: 113285
Ending Edges: 176449
iteration 6
Starting Nodes: 113285
Starting Edges: 176449
removing dead ends
Ending Nodes: 113189
Ending Edges: 176329
iteration 7
Starting Nodes: 113189
Starting Edges: 176329
removing dead ends
Ending Nodes: 113152
Ending Edges: 176281
iteration 8
Starting Nodes: 113152
Starting Edges: 176281
removing dead ends
Ending Node

In [17]:
extractor.save_as_shp(general, f'{city_name}/shp/simplified_topo_{city_name}')
#extractor.save_as_shp(merged_intersections, f'{city_name}/shp/merged_intersections_{city_name}')

u
v
key
geometry
length
highway
footway
bearing
foot
speed_kph
travel_time
grade
grade_abs
name
access
sidewalk


/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:279: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nodes.to_file(f"{self.DATA_BASE_PATH}/{path}_nodes.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'street_count' to 'street_cou'
  ogr_write(
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'aggr_node_number' to 'aggr_node_'
  ogr_write(
/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:281: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'tra

In [127]:
extractor.save_as_shp(general, f'{city_name}/shp/simplified_topo_{city_name}')

u
v
key
geometry
length
name
highway
sidewalk
bearing
foot
footway
speed_kph
travel_time
grade
grade_abs
access


/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:279: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nodes.to_file(f"{self.DATA_BASE_PATH}/{path}_nodes.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'street_count' to 'street_cou'
  ogr_write(
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'aggr_node_number' to 'aggr_node_'
  ogr_write(
/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:281: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'tra